In [117]:
import pandas as pd
import ast
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import bz2file as bz2

In [118]:
Data_1=pd.read_csv('tmdb_5000_movies.csv')

In [119]:
Data_2=pd.read_csv('tmdb_5000_credits.csv')

In [120]:
#for Poster
Data_3=pd.read_csv('movies_complete.csv')
Data_3=Poster[['title','poster_path']]

In [121]:
Data_1.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [122]:
movies=Data_2.merge(Data_1,on='title')
movies=pd.merge(movies,Data_3,on='title')

In [123]:
Movies=movies[['movie_id','title','cast','crew','genres','keywords','overview','poster_path']]

In [124]:
Movies.dropna(inplace=True)

In [125]:
Movies.drop_duplicates(inplace=True)

In [126]:
def name(x):
    y=[ ]
    x=ast.literal_eval(x)
    a=0
    for i in x:     
        k=i['name']
        #if a==3:
           # break
        #a+=1
        k=k.replace(' ','')
        k=k.lower()
        y.append(k)
    return(y)
        

In [127]:
Movies['cast']=Movies['cast'].apply(name)

In [128]:
Movies['genres']=Movies['genres'].apply(name)

In [129]:
Movies['keywords']=Movies['keywords'].apply(name)

In [130]:
Movies['overview']=Movies['overview'].apply(lambda x :x.split( ))
#Movies['overview']=Movies['overview'].apply(lambda x :x.lower())

In [131]:
def crews(y):
    T=[ ]
    y=ast.literal_eval(y)
    for i in y :
        #print(y)
        if(i['job']=='Director'):
            D=i['name']
            D=D.replace(' ','')
            D=D.lower()
            T.append(D)
    return(T)
        

In [132]:
Movies['crew']=Movies['crew'].apply(crews)

In [133]:
Tab=Movies['cast']+Movies['crew']+Movies['genres']+Movies['keywords']+Movies['overview']

In [134]:
Tab=Tab.apply(lambda x:' '.join(x))

In [135]:
Movies['Tab']=Tab

In [136]:
Movies=Movies[['movie_id', 'title','poster_path','Tab']]

In [137]:
ps=PorterStemmer()

In [138]:
def steamr(text):
    G=[]
    for i in text.split():
        i=i.lower()
        G.append(ps.stem(i))
        
    return ' '.join(G)        
    

In [139]:
Movies['Tab']=Movies['Tab'].apply(steamr)

In [143]:
CV=CountVectorizer(max_features=5000,stop_words='english')

In [144]:
Vector=CV.fit_transform(Movies['Tab']).toarray()

In [145]:
Simillier=cosine_similarity(Vector)

In [146]:
import pickle

In [147]:
#pickle.dump(Simillier,open('Movie_recommander_111.pkl','wb'))

In [148]:
pickle.dump(Movies,open('Data_Frame.pkl','wb'))

In [149]:
def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        pickle.dump(data, f)

In [150]:
compressed_pickle('Movie_Recommandtion_System', Simillier)

# END